In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


In [2]:
X, y = make_classification(n_samples=1000,n_features=10,n_informative=2,n_redundant=8,
                            weights=[0.9,0.1],flip_y=0,random_state=42)

np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

ExP:1 - Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1,solver='liblinear')
log_reg.fit(X_train,y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test,y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       270
           1       0.78      0.60      0.68        30

    accuracy                           0.94       300
   macro avg       0.87      0.79      0.82       300
weighted avg       0.94      0.94      0.94       300



ExP:2 - Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30,max_depth=3)
rf_clf.fit(X_train,y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       270
           1       0.89      0.53      0.67        30

    accuracy                           0.95       300
   macro avg       0.92      0.76      0.82       300
weighted avg       0.94      0.95      0.94       300



ExP:3- Train XgBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric='logloss')
xgb_clf.fit(X_train,y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.88      0.77      0.82        30

    accuracy                           0.97       300
   macro avg       0.93      0.88      0.90       300
weighted avg       0.97      0.97      0.97       300



ExP:4 -Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=0)
X_train_res,y_train_res = smt.fit_resample(X_train,y_train)

np.unique(y_train_res,return_counts=True)



(array([0, 1]), array([614, 614]))

In [8]:
xgb_clf =XGBClassifier(use_label_encoder = False,eval_metric='logloss')
xgb_clf.fit(X_train,y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.88      0.77      0.82        30

    accuracy                           0.97       300
   macro avg       0.93      0.88      0.90       300
weighted avg       0.97      0.97      0.97       300



Track Experiments using MLFLow

In [9]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1,solver='liblinear'),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30,max_depth=3),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False,eval_metric='logloss'),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False,eval_metric='logloss'),
        (X_train,y_train),
        (X_test,y_test)
    )
    
]

In [10]:
reports = []

for model_name,model,train_set,test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred,output_dict=True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [12]:
mlflow.set_experiment("imbalance classification")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i,element in enumerate(models):
    model_name =element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model",model_name)
        mlflow.log_metric("accuracy",report['accuracy'])
        mlflow.log_metric('recall_class_1',report['1']['recall'])
        mlflow.log_metric('recall_class_1',report['0']['recall'])
        mlflow.log_metric('f1_score_marco',report['macro avg']['f1-score'])
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model,"model")
        else:
            mlflow.sklearn.log_model(model,"model")

2025/11/05 18:09:34 INFO mlflow.tracking.fluent: Experiment with name 'imbalance classification' does not exist. Creating a new experiment.


2025/11/05 18:09:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/05 18:09:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/05 18:09:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/234559682912207288/runs/c7453c6c13c8430ca6cbc8693b57ea85
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/234559682912207288


2025/11/05 18:09:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/05 18:09:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/234559682912207288/runs/7db62e5973f34931905581825693026f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/234559682912207288


2025/11/05 18:10:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/05 18:10:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/234559682912207288/runs/f6fedd0ff4c44eb1911dbadfcae51a10
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/234559682912207288


2025/11/05 18:10:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/234559682912207288/runs/e6fd2bf8ddfb4399a329f431a8cd6f82
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/234559682912207288
